# Name matching

This notebook will run the name matching algorithm for an specific country

### Create `.egg` file from C++/Cython module

In [ ]:
import os

from glob import glob
from os import path

In [ ]:
! bash compile_library.sh

In [ ]:
egg_file = glob(path.join('cython', 'dist', '*.egg'))[0]
egg_file

**Spark cluster configuration**

We will be broadcasting a possible big sparse matrix, then it could be necessary to allocate up to 4-5GB of memory to each executor, the extreme cases are US or GB, for other countries this memory can be less.

In [ ]:
import findspark
findspark.init()

import pyspark

conf = pyspark.SparkConf().setAppName('NameMatching_Notebook')
conf.setExecutorEnv('PYTHON_EGG_CACHE', '/tmp')
# supposing you have 4 nodes with 8 cores each
# NOTE: HDInsight by default allocates a single core per executor
# regardless of what we set in here. The truth is in Yarn UI not in Spark UI.
conf.set('spark.dynamicAllocation.enabled', False)
conf.set('spark.executor.instances', 4) # low so that memory sharing is high
conf.set('spark.executor.cores', 14)
conf.set('spark.executor.memory', '14g')
conf.set('spark.driver.memory', '15g')

**get a spark context**

In [ ]:
sc = pyspark.SparkContext(conf=conf)
sqlContext = pyspark.sql.SQLContext(sc)

**send Cython code**

For doing the sparse matrix multiplication we are using c++ code wrapped using Cython so we need to send this file to every worker.

In [ ]:
sc.addPyFile(egg_file)

**imports**

In [ ]:
import sparse_dot_topn.sparse_dot_topn as ct # this is the cython module

In [ ]:
from pyspark import keyword_only

from pyspark.ml import Pipeline
from pyspark.ml import Transformer

from pyspark.ml.feature import CountVectorizer, HashingTF
from pyspark.ml.feature import IDF
from pyspark.ml.feature import NGram
from pyspark.ml.feature import Normalizer
from pyspark.ml.feature import RegexTokenizer
from pyspark.ml.feature import StopWordsRemover

from pyspark.mllib.linalg.distributed import IndexedRow
from pyspark.mllib.linalg.distributed import IndexedRowMatrix

from pyspark.ml.param.shared import HasInputCol
from pyspark.ml.param.shared import HasOutputCol
from pyspark.ml.param.shared import Param
from pyspark.ml.param.shared import Params

from pyspark.sql import functions as sf
from pyspark.sql.types import LongType
from pyspark.sql.types import StructField
from pyspark.sql.types import StructType
from pyspark.sql.types import StringType

from pyspark.sql.types import ArrayType, IntegerType, StringType, FloatType
from pyspark.mllib.linalg import VectorUDT, Vector, Vectors
from pyspark.mllib.linalg import SparseVector
from pyspark.sql.functions import udf
from pyspark.sql.types import StructField, StructType
from pyspark.sql.functions import array, struct

import numpy as np
from scipy.sparse import csr_matrix, coo_matrix
from scipy.sparse import _sparsetools
from scipy.sparse.sputils import get_index_dtype
import math
from itertools import tee

**Cython sparse multiplication**

In [ ]:
def chunk_dot_limit(A, B, ntop, threshold=0, start_row=0):
    """Sparse dot product of two matrices AxB
    
    It will return the upper triangular matrix.
    
    Args:
        A (csr matrix): Left matrix with shape m x k
        B (csr matrix): Right matrix with shape k x n
        ntop (int): Get only n most similar
        threshold (float): Return similarities above this number
        start_row (int): Set start row index for matrix A
    """
    B = B.tocsr()
    
    M = A.shape[0]
    N = B.shape[1]

    idx_dtype = np.int32

    # maximum number of possible non-zero element in the upper triangular matrix
    nnz_max = min(M * (2 * (N - start_row) - M - 1) / 2, M * ntop)

    # the arrays will be returned by reference
    rows = np.empty(nnz_max, dtype=idx_dtype)
    cols = np.empty(nnz_max, dtype=idx_dtype)
    data = np.empty(nnz_max, dtype=A.dtype)

    # number of found non-zero entries in the upper triangular matrix
    # I'll use this value to slice the returning numpy array
    nnz = ct.sparse_dot_topn(
        M, N,
        np.asarray(A.indptr, dtype=idx_dtype),
        np.asarray(A.indices, dtype=idx_dtype),
        A.data,
        np.asarray(B.indptr, dtype=idx_dtype),
        np.asarray(B.indices, dtype=idx_dtype),
        B.data,
        ntop,
        threshold, rows, cols, data, start_row)
    
    return ((int(i), int(j), float(v)) for i, j, v in zip(rows[:nnz], cols[:nnz], data[:nnz]))

## Input parameters

In [ ]:
country_code = 'DK'
fraction = 1. # use only this fraction of the data
threshold = 0.9 # ignore similarities below threshold
ntop = 1000 # max number of similarities to match per name
n_gram = 2 # size of letter n-grams to form
vocabulary_size = 15000 # number of ngrams to take
min_appearances = 2 # keep n-grams that appear at least this number of times in the corpus
input_path = 'adl://ulohubdldevne.azuredatalakestore.net/data/parquet/OPERATORS.parquet'
output_path = 'adl://ulohubdldevne.azuredatalakestore.net/data/parquet/OPERATORS_MATCHED.parquet'

## Load operator data

- Load complete data
- Get only the one for the specified country
- Drop `NA` in column `NAME_CLEANSED`
- Create unique `id` column
- Create `name` column to use for deduplication
- Clean `name` column
- Create `dummy_id` columns which represent the row number

In [ ]:
operators = sqlContext.read.parquet(input_path)

In [ ]:
schema = StructType([StructField("name_index", LongType(), False),
                     StructField("id", StringType(), False),
                     StructField("name", StringType(), False)])

In [ ]:
drop_chars = "\\\\!#%&()*+-/:;<=>?@\\^|~\u00A8\u00A9\u00AA\u00AC\u00AD\u00AF\u00B0\u00B1\u00B2\u00B3\u00B6\u00B8\u00B9\u00BA\u00BB\u00BC\u00BD\u00BE\u2013\u2014\u2022\u2026\u20AC\u2121\u2122\u2196\u2197\u247F\u250A\u2543\u2605\u2606\u3001\u3002\u300C\u300D\u300E\u300F\u3010\u3011\uFE36\uFF01\uFF06\uFF08\uFF09\uFF1A\uFF1B\uFF1F{}\u00AE\u00F7\u02F1\u02F3\u02F5\u02F6\u02F9\u02FB\u02FC\u02FD\u1BFC\u1BFD\u2260\u2264\u2DE2\u2DF2\uEC66\uEC7C\uEC7E\uED2B\uED34\uED3A\uEDAB\uEDFC\uEE3B\uEEA3\uEF61\uEFA2\uEFB0\uEFB5\uEFEA\uEFED\uFDAB\uFFB7\u007F\u24D2\u2560\u2623\u263A\u2661\u2665\u266A\u2764\uE2B1\uFF0D"
regex = "[{}]".format(drop_chars)

In [ ]:
country_opr = (
    operators[operators['COUNTRY_CODE'] == country_code]
    .na.drop(subset=['NAME_CLEANSED'])
    .sample(False, fraction)
    .withColumn('id', sf.concat('COUNTRY_CODE', sf.lit("~"), 'SOURCE', sf.lit('~'), 'REF_OPERATOR_ID'))
    .fillna('')
#     .withColumn('name', sf.concat('NAME_CLEANSED', sf.lit(' '), sf.col('CITY_CLEANSED'), sf.lit(' '), sf.col('STREET_CLEANSED'), sf.lit(' '), sf.col('ZIP_CODE_CLEANSED')))
    .withColumn('name', sf.concat('NAME_CLEANSED', sf.lit(' '), sf.col('CITY_CLEANSED')))
    .withColumn('name', sf.regexp_replace('name', regex, ''))
    .withColumn('name', sf.trim(sf.regexp_replace('name', '\s+', ' ')))
    .select('id', 'name')
    .sort('id', ascending=True)
    .rdd
    .zipWithIndex()
    .map(lambda x: (x[1], x[0][0], x[0][1]))
    .toDF(schema)
)
country_opr.persist()

print("Nr Rows:", country_opr.count())
country_opr.show(5, truncate=False)

## Transformation pipeline

1. `RegexTokenizer` to separate each character into an array
2. `NGram` to create n-grams from the letters
3. `CountVectorizer` to count number of occurences of any n-gram in each name (this can be changed at some point to `HashingTF`)
4. `IDF` perform inverse document frequency of the n-grams created
5. `Normalizer` to L2 normalize the resulting tfidf vectors

In [ ]:
# each character into a vector
regexTokenizer = RegexTokenizer(inputCol="name", outputCol="tokens", pattern="")
# create vector of n-grams
ngram_creator = NGram(n=n_gram, inputCol="tokens", outputCol="n_grams")
# term frequency
tf_counter = CountVectorizer(minTF=1, minDF=min_appearances, vocabSize=vocabulary_size, 
                             inputCol='n_grams', outputCol='term_frequency', binary=False)

# tf_counter = HashingTF(numFeatures=64, 
#                        inputCol='n_grams', outputCol='term_frequency', binary=False)

# inverse document frequency
idf_counter = IDF(inputCol="term_frequency", outputCol="tfidf_vector")
# L2 normalization of vector
l2_normalizer = Normalizer(inputCol="tfidf_vector", outputCol="name_vector", p=2)

tfidf_vectorizer = Pipeline(stages=[regexTokenizer, ngram_creator, tf_counter, idf_counter, l2_normalizer])

In [ ]:
encoded_names = tfidf_vectorizer.fit(country_opr).transform(country_opr).select(['name_index', 'name_vector'])
print("dummy_id:\t row number\nname_vector:\t (nr_ngrams,[indices], [values])")
encoded_names.show(5, truncate=True)

## Get all index comparisons to be made

Transform to a dataframe which contains a column with the name index another with the n-gram index and a third one with the value given by the tfidf-normalizer transformation above.

1. Unpack sparse vector into a lit of tuples containing (index, value) for value bigger than 0.1
2. Set each tuple value to its own row according to the correspoing name_index
3. Separte these tuples into different columns
4. Get columns with name index, ngram index and the corresponding value.

In [ ]:
schema_sparse = ArrayType(StructType([
    StructField("ngram_index", IntegerType(), False),
    StructField("value", FloatType(), False)
]))

In [ ]:
# @sf.udf(schema_sparse)
def unpack_vector(sparse):
    """Combine indices and values into a tuples.
    
    For each value below 0.1 in the sparse vector we create a tuple and
    then add these tuples into a single list. The tuple contains the
    index and the value.  
    """
    return ((int(index), float(value)) for index, value in zip(sparse.indices, sparse.values) if value > 0.1)

udf_unpack_vector = sf.udf(unpack_vector, schema_sparse)


names_vs_ngrams = (
    encoded_names
    .withColumn('explode', sf.explode(udf_unpack_vector(sf.col('name_vector'))))
    .withColumn('ngram_index',  sf.col('explode').getItem('ngram_index'))
    .withColumn('value',  sf.col('explode').getItem('value'))
    .select('name_index', 'ngram_index', 'value')
)

names_vs_ngrams.persist()

names_vs_ngrams.show(5, truncate=False)

Summary about the obtained values

In [ ]:
names_vs_ngrams.describe('value').show()

## Transform into Local Scipy CSR matrix

1. Load pyspark into pandas dataframe (this step might need lots of memory)
2. Change column types to match c++ implementation
2. Create CSR Scipy matrix using the index and the values

In [ ]:
df = names_vs_ngrams.toPandas()

In [ ]:
df.name_index = df.name_index.astype(np.int32)
df.ngram_index = df.ngram_index.astype(np.int32)
df.value = df.value.astype(np.float64)

names_vs_ngrams.unpersist()
df.head(5)

Create CSR compressed form matrix

In [ ]:
csr_names_vs_ngrams = csr_matrix(
    (df.value.values, (df.name_index.values, df.ngram_index.values)),
    shape=(df.name_index.max() + 1, df.ngram_index.max() + 1),
    dtype=np.float64)
del df

print("Matrix Shape:\t", csr_names_vs_ngrams.shape, "=", csr_names_vs_ngrams.shape[0] * csr_names_vs_ngrams.shape[1])
print("NonZero:\t", csr_names_vs_ngrams.count_nonzero())
print("Sparsity:\t", csr_names_vs_ngrams.count_nonzero() / (csr_names_vs_ngrams.shape[0] * csr_names_vs_ngrams.shape[1]))

## Broadcast and parallelize

1. Broadcast complete matrix
2. Calculate maximum possible number of partitions (no single row matrix partitions, we set minimum of 500 rows)
3. Parallelize chunks of the matrix

In [ ]:
m_T_bcast = sc.broadcast(csr_names_vs_ngrams.transpose())

In [ ]:
max_n_chunks = math.floor(csr_names_vs_ngrams.shape[0] / 500)
max_n_chunks

when we parallelize we also send the row number as this will be important when trying to compare with the real names of getting their id.

In [ ]:
n_chunks = min(56 * 4, max_n_chunks)

chunk_size = math.ceil(csr_names_vs_ngrams.shape[0] / n_chunks)
n_chunks = math.ceil(csr_names_vs_ngrams.shape[0] / chunk_size)
chunks = [(csr_names_vs_ngrams[(i * chunk_size) : min((i + 1) * chunk_size, csr_names_vs_ngrams.shape[0])], i * chunk_size) for i in range(n_chunks)]
m_distr = sc.parallelize(chunks, numSlices=n_chunks)

print("Nr. Chunks:\t", len(chunks))
print("Chunk Size:", chunk_size, "from:", csr_names_vs_ngrams.shape[0])

## Execute sparse multiplication

It is important to remember that the index for the matrices is always from 0 to the number of rows - 1, hence we need to pass the starting row number.

In [ ]:
similarity = m_distr.flatMap(lambda x: chunk_dot_limit(x[0],
                                                       m_T_bcast.value,
                                                       ntop=ntop,
                                                       threshold=threshold,
                                                       start_row=x[1]))

sim_schema = StructType([
    StructField("i", IntegerType(), False),
    StructField("j", IntegerType(), False),
    StructField("SIMILARITY", FloatType(), False)
])

similarity = similarity.toDF(sim_schema)

In [ ]:
del csr_names_vs_ngrams

In [ ]:
from pyspark.sql.window import Window
# group similarities with column i being the group id
grouping_window = (Window
                   .partitionBy('j')
                   .orderBy(sf.asc('i')))
# keep only the first entry sorted alphabetically
grp_sim = (
    similarity
    .withColumn("rn", sf.row_number().over(grouping_window))
    .filter(sf.col("rn") == 1)
    .drop('rn')
)
# remove group ID from column j
grp_similarity = grp_sim.join(
    grp_sim.select('j').subtract(grp_sim.select('i')),
    on='j', how='inner'
)

## Join matches with original information of the names

In [ ]:
matches = (
    grp_similarity
    .join(country_opr, grp_similarity['i'] == country_opr['name_index'],
          how='left').drop('name_index')
    .selectExpr('i', 'j', 'id as SOURCE_ID', 'SIMILARITY', 'name as SOURCE_NAME')
    .join(country_opr, grp_similarity['j'] == country_opr['name_index'],
          how='left').drop('name_index')
    .withColumn('COUNTRY_CODE', sf.lit(country_code))
    .selectExpr('COUNTRY_CODE', 'SOURCE_ID', 'id as TARGET_ID',
                'SIMILARITY', 'SOURCE_NAME', 'name as TARGET_NAME')
)

In [ ]:
%%time
matches.persist()

In [ ]:
m_T_bcast.unpersist()

## Results exploration

are there any similarities with itself?

In [ ]:
matches[matches['SOURCE_ID'] == matches['TARGET_ID']].count()

are there any permuted similarities?

In [ ]:
(matches
 .withColumn('original', sf.concat('SOURCE_ID', 'TARGET_ID'))
 .withColumn('permuted', sf.concat('TARGET_ID', 'SOURCE_ID'))
 .filter(sf.col('original') == sf.col('permuted'))
 .count())

number of results

In [ ]:
matches.count()

summary of similarities found

In [ ]:
matches.describe('SIMILARITY').show()

how many matches per name?

Ideally we would like this to be less than the set number of maximum number of similarities to find `ntop`

In [ ]:
print("The set maximum number of matches is:", ntop)

matches.groupBy(['SOURCE_ID', 'SOURCE_NAME']).count().sort('count', ascending=False).show(4)

**Matches for a short name**

In [ ]:
cut_off = 10

In [ ]:
matches_small = matches.where(sf.size(sf.split('SOURCE_NAME', '')) < cut_off)
matches_small.describe('SIMILARITY').show()
matches_small.select('SIMILARITY', 'SOURCE_NAME', 'TARGET_NAME').show(20, truncate=False)

**Matches for long names**

In [ ]:
matches_big = matches.where(sf.size(sf.split('SOURCE_NAME', '')) > cut_off)
matches_big.describe('SIMILARITY').show()
matches_big.select('SIMILARITY', 'SOURCE_NAME', 'TARGET_NAME').show(20, truncate=False)

## Extra Check

In [ ]:
from pyspark.sql.window import Window

window = Window.partitionBy('TARGET_ID').orderBy(sf.desc('SIMILARITY'), sf.asc('SOURCE_ID'))
window = Window.partitionBy('TARGET_ID').orderBy(sf.asc('SOURCE_ID'))

In [ ]:
single_matches = (matches
                  .withColumn("rn", sf.row_number().over(window))
                  .filter(sf.col("rn") == 1).drop('rn')
                  .filter(sf.col('TARGET_ID') > sf.col('SOURCE_ID'))
                  .sort('SOURCE_ID', ascending=True))
single_matches.show(20, truncate=False)

In [ ]:
res = single_matches.join(single_matches.select('TARGET_ID').subtract(single_matches.select('SOURCE_ID')), on='TARGET_ID', how='inner')
# res.show()
res[res.SOURCE_NAME.like('%aarhus tec%') | res.TARGET_NAME.like('%aarhus tec%')].sort('SOURCE_ID').select('SOURCE_ID', 'TARGET_ID', 'SIMILARITY', 'SOURCE_NAME', 'TARGET_NAME').show(100, truncate=False)

In [ ]:
(res[(matches['SOURCE_NAME'] == 'aarhus tech') | (res['TARGET_NAME'] == 'aarhus tech')]
 .select('SOURCE_ID', 'TARGET_ID', 'SIMILARITY', 'SOURCE_NAME', 'TARGET_NAME')
).show(200, truncate=False)

In [ ]:
res[res['TARGET_ID'] == 'DK~MM-INIT-OPER~O~1368026'].count()

In [ ]:
res[res['SOURCE_ID'] == 'DK~MM-INIT-OPER~O~1368026'].show()